In [23]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()


from urllib.request import urlopen

La solicitud de cabecera del Agente de Usuario contiene una cadena característica que permite identificar el protocolo de red  que ayuda a descubrir  el tipo de aplicación, sistema operativo, provedor del software o laversión del software de la petición del agente de usuario.

In [24]:
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})

Tenemos un objeto Response. Podemos obtener toda la información que necesitamos a partir de este objeto.

In [25]:
sapo = "https://casa.sapo.pt/comprar-apartamentos/cascais/"

response = get(sapo, headers = headers)

In [58]:
print(response) # un 200 es una buena señal :D

<Response [200]>


response.text returns the content of the response, in unicode. Basically, it refers to Binary Response content.
this response object would be used to access certain features such as content, headers, etc.

In [27]:
print(response.text[:1000])



<!DOCTYPE html>

<html lang="pt">
<head><title>
	Casas para Venda, Apartamentos em Cascais, CASA SAPO - Portal Nacional de Imobiliário
</title><meta name="author" content="CASA SAPO - Portal Nacional de Imobiliário - Janela Digital SA" />
    <meta name="application-name" content="CASA SAPO - Portal Nacional de Imobiliário" data-copyright="Janela Digital SA" data-generated-time="15/10/2020 19:17" />
    
<meta name="content-language" content="pt" />
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<meta name="description" content="Casas para Venda, 3020 Apartamentos em Cascais, Deseja comprar casa? No maior Portal Imobiliário Nacional temos milhares de apartamentos e moradias em Lisboa, no Porto e por todo o país." />
<meta name="keywords" content="Casas para Venda, Apartamentos em Cascais, venda, compra, comprar, casas, imóveis, apartamentos, moradias, terrenos, porto, lisboa" />
<meta name="referrer" content="always" />
<link rel="alternate" t


In [28]:
html_soup = BeautifulSoup(response.text, 'html.parser')

Beautiful Soup picks the text from the response and parses the information in a way that makes it easier for you to navigate in its structure and get its contents.

In [29]:
house_containers = html_soup.find_all('div', class_= "searchResultProperty")

findall() finds *all* the matches and returns them as a list of strings, with each string representing one match. 

In [30]:
first = house_containers[0]
first.find_all("span")

[<span class="titleG3">Apartamento T3 Sao Domingos Rana</span>,
 <span>
                         Apartamento T3, Caparide, São Domingos de Rana, Cascais, Lisboa
                     </span>,
 <span class="btnContactPVPI" id="MC_PropertyInList_repProperties_btnContactPVPINormal_0" onclick="ShowContactForm('f48f5a18-a95d-11e9-980d-060000000052', '13', '5', true, true, '677040'); return false;" style="z-index: 9999;" title="Contacte Anunciante">Contacte Anunciante</span>,
 <span>360 000 <strong title="Euro">€</strong></span>]

First va a tener la estructura de nuestra primera casa, sacada de house_containers

### Obtenemos el precio de la vivienda

In [32]:
var_1 = first.find_all("span")[3].text
var_1

'360\xa0000 €'

In [33]:
var_1 = var_1.replace('\xa0', '')
var_1

'360000 €'

In [34]:
# Iteramos con el itertools. Takewhile se queda con aquello que es true (is digit devuelve true/false)
var_1 = int(''.join(itertools.takewhile(str.isdigit, var_1)))

In [35]:
print(var_1, type(var_1))

360000 <class 'int'>


### Obtenemos características de la casa

In [36]:
#Location
location = first.find_all('p', class_ = "searchPropertyLocation")[0].text
location

'\r\n                    Caparide, São Domingos de Rana, Cascais, Lisboa\r\n                '

In [41]:
#Find() devuelve el índice del caracter que encuentra
# si no encuentra devuelve -1
location[22:location.find(',')]

'Caparide'

In [42]:
#tamaño en m2
first.find_all('p')[7]

<p>93m²</p>

In [43]:
#Obtener fecha de publicación del anuncio
first.find_all('div', class_="searchPropertyDate")

[]

In [87]:
#Fecha del post
#pd.to_datetime(first.find_all('div', class_="searchPropertyDate")[0].text[3:10])

In [64]:
#Short description
first.find_all('p', class_="searchPropertyDescription")[0].text[22:-25]

'Apartamento T3 novo em CAPARIDE em prédio com projecto de arquitectura contemporânea, num condomínio intimista, com 4 apartamentos T3. Situado em São Domingos de Rana, em zona calma e bem localizado a 5 Km das praias de Sã'

Estos ejemplos deberían ser suficientes para que hagamos nuestra propia investigación. Se aprende mucho con solo probar con la estructura html y manipular los valores que se devuelven, hasta que obtenemos lo que queremos.

### Continuamos cogiendo todos los links

In [45]:
# Para coger todos los links
for url in first.find_all('a'):
    print(url.get('href'))

https://gespub.casa.sapo.pt/v3/webinterface/client/counter.aspx?p=677040,677039&c=1&MCA=1&TW=152&BU=1&l=https%3a%2f%2fcasa.sapo.pt%2fcomprar-apartamento-t3-cascais-sao-domingos-de-rana-caparide-f48f5a18-a95d-11e9-980d-060000000052.html%3fg3pid%3d677040
https://gespub.casa.sapo.pt/v3/webinterface/client/counter.aspx?p=677040,677039&c=1&MCA=1&TW=152&BU=1&l=https%3a%2f%2fcasa.sapo.pt%2fcomprar-apartamento-t3-cascais-sao-domingos-de-rana-caparide-f48f5a18-a95d-11e9-980d-060000000052.html%3fg3pid%3d677040
/agencia/afinest-luxury-real-estate/?cl=11398&sys=5
https://gespub.casa.sapo.pt/v3/webinterface/client/counter.aspx?p=677040,677039&c=1&MCA=1&TW=152&BU=1&l=https%3a%2f%2fcasa.sapo.pt%2fcomprar-apartamento-t3-cascais-sao-domingos-de-rana-caparide-f48f5a18-a95d-11e9-980d-060000000052.html%3fg3pid%3d677040
https://gespub.casa.sapo.pt/v3/webinterface/client/counter.aspx?p=677040,677039&c=1&MCA=1&TW=152&BU=1&l=https%3a%2f%2fcasa.sapo.pt%2fcomprar-apartamento-t3-cascais-sao-domingos-de-rana-capa

In [59]:
#Obtener todos los links
#first.find_all('a')[0].get('href')[0:-6]

In [47]:
# Creamos una lista que va a formar a nuestro dataframe
titles = []
created = []
prices = []
areas = []
zone = []
condition = []
descriptions = []
urls = []

In [51]:
#%%time

n_pages = 0

for page in range (0,10):
    n_pages +=1
    sapo_url = "https://casa.sapo.pt/comprar-apartamentos/cascais/"
    r = get(sapo_url, headers = headers)
    page_html = BeautifulSoup(r.text, 'html.parser')
    house_containers = page_html.find_all('div', class_ = "searchResultProperty")
    
    if house_containers != []:
        for container in house_containers:
            try:
                
            
            
            #Precio 
                price = container.find_all('span')[4].text
                print(price)
                if price == 'Contacte Anunciante':
                    price = container.find_all('span')[3].text

                    if price.find('/') != -1:
                        price = price[0:price.find('/')-1]

                if price.find('/') != -1:
                    price = price[0:price.find('/')-1]
                print(price)    
                price_ = [int(price[s]) for s in range(0, len(price)) if price[s].isdigit()]

                price = ''
                print(price_)
                for x in price_:
                    price = price + str(x)
                prices.append(int(price))
            
            except:
                continue
            
            #Zona
            location = container.find_all('p', class_="searchPropertyLocation")[0].text
            location = location[22:location.find(',')]
            zone.append(location)
            
            #Title
            name = container.find_all('span')[0].text
            titles.append(name)
            
            #Description
            desc = first.find_all('p', class_="searchPropertyDescription")[0].text[22:-25]
            descriptions.append(desc)
            
            #Url
            link = container.find_all('a')[0].get('href')[0:-6]
            urls.append(link)
            
    else:
        break
        
    #sleep(randint(1,2))
    
print('You scraped {} pages containing {} properties.'.format(n_pages, len(titles)))   

You scraped 1 pages containing 9481 properties.


In [55]:
cols = ['Title', 'Price', 'Zone',  'Description', 'URL']

cascais = pd.DataFrame({'Title': titles,
                           'Price': prices,
                           'Zone': zone,
                           'Description': descriptions,
                           'URL': urls})[cols]

cascais.to_excel('lisboa_raw.xls')

cascais = pd.read_excel('lisboa_raw.xls')

In [65]:
cascais.head()

,Unnamed: 0,Title,Price,Zone,Description,URL
0,0,Excelente Apartamento T3 Novo com terraço e vi...,3,Quinta das Marianas (Parede),Apartamento T3 novo em CAPARIDE em prédio com ...,https://gespub.casa.sapo.pt/v3/webinterface/cl...
1,1,T2 renovado no Estoril,390000,Cascais e Estoril,Apartamento T3 novo em CAPARIDE em prédio com ...,https://gespub.casa.sapo.pt/v3/webinterface/cl...
2,2,T2 Jardins da Parede com Terraço,520000,Jardins da Parede (Parede),Apartamento T3 novo em CAPARIDE em prédio com ...,https://gespub.casa.sapo.pt/v3/webinterface/cl...
3,3,T3 no Cascais Park,650000,Cascais e Estoril,Apartamento T3 novo em CAPARIDE em prédio com ...,https://gespub.casa.sapo.pt/v3/webinterface/cl...
4,4,Apartamento T3 em Condomínio na Costa da Guia,730000,Costa da Guia (Cascais),Apartamento T3 novo em CAPARIDE em prédio com ...,https://gespub.casa.sapo.pt/v3/webinterface/cl...
